# Лабораторная работа №4
# Задача класссификации

Пусть имеется множество объектов, характеризующихся признаками и разделённых некоторым образом на классы.

Задача классификации — это задача построения алгоритма (функции), способного 
классифицировать произвольный объект из исходного пространства признаков, т.е. определять метку класса для этого объекта.


## Загрузка наборов данных

Некоторые наиболее часто используемые в учебных и исследовательских целях наборы данных можно загрузить напрямую из бибиотеки scikit-learn. Это относится к таким наборам, как

* boston house-prices dataset (regression)
* iris dataset (classification)  
* diabetes dataset (regression)  
* digits dataset (classification)  
* physical excercise linnerud dataset  
* wine dataset (classification)  
* breast cancer wisconsin dataset (classification)  


Например, для загрузки набора "Ирисы" можно использовать код:

In [ ]:
from sklearn import datasets

iris = datasets.load_iris()

Набор данных загружается в словарь специального вида со следующими ключами: 

In [ ]:
iris.keys()

In [ ]:
print(iris.DESCR)

Названия признаков и меток классов находятся в элементах `feature_names` и `target_names`:

In [ ]:
print( "Признаки: ", iris['feature_names'] )
print( "Метки: ", iris.target_names )

Значения признаков и меток хранятся в элементах `data` и `target` как массивы `ndarray`:

In [ ]:
print(iris.data[0:5])
print(iris.target[0:5])

Если вызвать метод `load_iris()` с ключом `as_frame=True`, то набор данных будет доступен как объект `DataFrame`: 

In [ ]:
iris2 = datasets.load_iris(as_frame=True)
type(iris2.frame)

## Препроцессинг наборов данных

Во многих алгоритмах классификации содержатся допущения относительно классифицируемых данных, поэтому в ряде случаев может потребоваться предварительная подготовка (препроцессинг) данных. 

### Масштабирование данных

Если данные содержат признаки разного масштаба, то многие алгоритмы классификации работают лучше, когда признаки имеют одинаковый масштаб. В частности, это полезно для алгоритмов, использующих меры расстояния, такие как метод k-ближайших соседей. 

Создадим случайный набор точек, преобразуем целочисленный массив `X` к типу `float`, масштабируем его на интервал `[0, 1]` и визуализируем на плоскости:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

X = np.random.randint(0, 100, (50, 2))

In [ ]:
X = np.array(X, dtype=float)

In [ ]:
# масштабирование на [0,1]
X[:,0] = (X[:,0] - np.min(X[:,0])) / (np.max(X[:,0]) - np.min(X[:,0]))
X[:,1] = (X[:,1] - np.min(X[:,1])) / (np.max(X[:,1]) - np.min(X[:,1]))

In [ ]:
plt.scatter(X[:,0], X[:,1])
plt.axis('square')
plt.show()

Можно масштабировать данные, используя класс `MinMaxScaler` из scikit-learn:

In [ ]:
from sklearn.preprocessing import MinMaxScaler

np.set_printoptions(precision=3)

X = iris.data
scaler = MinMaxScaler(feature_range=(1, 5)) # значения признаков от 1 до 5
rescaledX = scaler.fit_transform(X)

print(rescaledX[0:5,:])

### Стандартизация данных

Стандартизация - это метод преобразования признаков к виду, когда они имеют среднее значение 0 и стандартное отклонение 1. Этот способ подходит, в частности, для методов машинного обучения, предполагающих нормальное распределение входных данных. 

Создадим случайный набор точек, преобразуем целочисленный массив `X` к типу `float`, стандартизуем его и визуализируем на плоскости:

In [ ]:
X = np.random.randint(0, 100, (50, 2))

In [ ]:
X = np.array(X, dtype=float)

In [ ]:
# стандартизация
X[:,0] = (X[:,0] - np.mean(X[:,0])) / np.std(X[:,0])
X[:,1] = (X[:,1] - np.mean(X[:,1])) / np.std(X[:,1])

In [ ]:
plt.scatter(X[:,0], X[:,1])
plt.axis('Equal')
plt.show()

In [ ]:
np.mean(X[:,0]), np.std(X[:,0]), np.mean(X[:,1]), np.std(X[:,1])

Набор данных можно стандартизовать при помощи класса `StandardScaler` из scikit-learn:

In [ ]:
from sklearn.preprocessing import StandardScaler

X = iris.data
scaler = StandardScaler().fit(X)
rescaledX = scaler.transform(X)

print(rescaledX[0:5,:])

### Нормировка данных

Нормировка - это изменение масштаба каждой строки (записи) до единичной длины. Метод полезен для разреженных наборов данных (со многими нулями) при использовании
алгоритмов, использующих расстояние (например, метод k-ближайших соседей). В качестве нормы можно использовать ‘l1’, ‘l2’ или ‘max’ (по умолчанию ‘l2’). 
Можно нормализовать данные с помощью класса `Normalizer` из scikit-learn: 

In [ ]:
from sklearn.preprocessing import Normalizer

X = iris.data
scaler = Normalizer(norm='max').fit(X)
normalizedX = scaler.transform(X)

print(normalizedX[0:5,:])

## Обучающая и контрольная (тестовая) выборки

Для оценки качества обученной модели классификации используют разбиение на обучающую (training) и тестовую (test) выборки. 

In [ ]:
X = iris.data
y = iris.target
X.shape, y.shape

In [ ]:
y

In [ ]:
shuffled_indexes = np.random.permutation(len(X))
shuffled_indexes

In [ ]:
test_ratio = 0.2
test_size = int(len(X) * test_ratio)

In [ ]:
test_indexes = shuffled_indexes[:test_size]
train_indexes = shuffled_indexes[test_size:]

In [ ]:
X_train = X[train_indexes]
y_train = y[train_indexes]

X_test = X[test_indexes]
y_test = y[test_indexes]

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

Объединим использованный выше код в одну функцию:

In [ ]:
def my_train_test_split(X, y, test_ratio=0.2, seed=None):
    """returns X_train, X_test, y_train, y_test"""
    assert X.shape[0] == y.shape[0], \
        "the size of X must be equal to the size of y"
    assert 0.0 <= test_ratio <= 1.0, \
        "test_ration must be valid"

    if seed:
        np.random.seed(seed)

    shuffled_indexes = np.random.permutation(len(X))

    test_size = int(len(X) * test_ratio)
    test_indexes = shuffled_indexes[:test_size]
    train_indexes = shuffled_indexes[test_size:]

    X_train = X[train_indexes]
    y_train = y[train_indexes]

    X_test = X[test_indexes]
    y_test = y[test_indexes]

    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = my_train_test_split(X, y)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

Разбиение на обучающую и тестовую выборки с аналогичным интерфейсом реализовано в `scikit-learn`:

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=666)

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

## Классификация методом K ближайших соседей

В методе ближайших соседей точка классифицируется согласно классам ее ближайших соседей.

Для иллюстрации метода рассмотрим следующий набор из 10 точек на плоскости:

In [ ]:
raw_data_X = [[3.393533211, 2.331273381],
              [3.110073483, 1.781539638],
              [2.280362439, 2.866990263],
              [1.343808831, 3.368360954],
              [3.582294042, 4.679179110],
              [7.423436942, 4.696522875],
              [5.745051997, 3.533989803],
              [9.172168622, 2.511101045],
              [7.792783481, 3.424088941],
              [7.939820817, 0.791637231]
             ]
raw_data_y = [0, 0, 0, 0, 1, 1, 1, 2, 2, 2]

In [ ]:
X_train = np.array(raw_data_X)
y_train = np.array(raw_data_y)

In [ ]:
plt.scatter(X_train[y_train==0,0], X_train[y_train==0,1], color='r')
plt.scatter(X_train[y_train==1,0], X_train[y_train==1,1], color='g')
plt.scatter(X_train[y_train==2,0], X_train[y_train==2,1], color='b')
plt.show()

Попробуем классифицировать по ближайщим соседям следующую точку `x`.

In [ ]:
x = np.array([6.5, 3.])

plt.scatter(X_train[y_train==0,0], X_train[y_train==0,1], color='r')
plt.scatter(X_train[y_train==1,0], X_train[y_train==1,1], color='g')
plt.scatter(X_train[y_train==2,0], X_train[y_train==2,1], color='b')
plt.scatter(x[0], x[1], color='y', edgecolor='k', s=200)
for idx in range(len(y_train)):
    plt.text(X_train[idx,0]+0.1, X_train[idx,1], idx, fontsize=12, color='k')
plt.show()

В качестве расстояния будем использовать евклидово расстояние на плоскости:

In [ ]:
distances = []

for x_train in X_train:
    d = np.sqrt(np.sum((x_train - x)**2))
    distances.append(d)

In [ ]:
distances = [np.sqrt(np.sum((x_train - x)**2)) for x_train in X_train] # list comprehension

In [ ]:
np.argsort(distances) # индексы отсортированного списка (по возрастанию)

Будем выбирать класс точки `x` по `k` ближайщим соседям:

In [ ]:
nearest = np.argsort(distances)
k = 5

In [ ]:
topK_y = [y_train[neighbor] for neighbor in nearest[:k]]
topK_y

In [ ]:
from collections import Counter

votes = Counter(topK_y)
votes

In [ ]:
votes.most_common(1)

In [ ]:
predict_y = votes.most_common(1)[0][0]
predict_y

Объединим программный код, использованный выше в одну функцию:

In [ ]:
#import numpy as np
#from collections import Counter


def kNN_classify(k, X_train, y_train, x):

    assert 1 <= k <= X_train.shape[0], "k must be valid"
    assert X_train.shape[0] == y_train.shape[0], \
        "the size of X_train must equal to the size of y_train"
    assert X_train.shape[1] == x.shape[0], \
        "the feature number of x must be equal to X_train"

    distances = [np.sqrt(np.sum((x_train - x)**2)) for x_train in X_train]
    nearest = np.argsort(distances)

    topK_y = [y_train[i] for i in nearest[:k]]
    votes = Counter(topK_y)

    return votes.most_common(1)[0][0]

In [ ]:
kNN_classify(k, X_train, y_train, x)

Аналогичные результаты дает классификатор метода ближайших соседей из библиотеки scikit-learn:

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

kNN_clf = KNeighborsClassifier(n_neighbors=5) # создаем классификатор
kNN_clf.fit(X_train, y_train)                 # обучаем классификатор

Для классификации точки `x` необходимо преобразование размеров (из вектора в матрицу):

In [ ]:
x

In [ ]:
x.reshape(1,-1)

In [ ]:
x.reshape(-1,1)

Одна из размерностей `reshape` может быть указана как -1, тогда значение этой размерности выводится из длины массива и других размерностей.

Прогнозируем класс точки `x` при помощи 

In [ ]:
kNN_clf.predict(x.reshape(1,-1))

In [ ]:
y_predict = kNN_clf.predict(x.reshape(1,-1))[0]
y_predict

Получаем для точки `x` ту же метку класса `2`.

## Классификация набора данных digits

Набор данных `digits` представляет собой изображения цифр от 0 до 9.

In [ ]:
digits = datasets.load_digits()

X = digits.data
y = digits.target
X.shape, y.shape

Метки классов представляют собой цифры от 0 до 9:

In [ ]:
y[:50]

А данные представляют собой изображения цифр:

In [ ]:
a_digit = X[666]
a_digit_image = a_digit.reshape(8, 8)

In [ ]:
plt.imshow(a_digit_image, cmap = plt.cm.binary)
plt.show()

Создадим и обучим классификатор метода ближайщих соседей на обучающей выборке из набора `digits`: 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

knn = KNeighborsClassifier(n_neighbors=3) # по трем ближайшим соседям
knn.fit(X_train,y_train)

Выполним прогнозирование меток классов (цифр):

In [ ]:
y_pred = knn.predict(X_test)

Оценим качество прогноза при помощи показателя "доля верных ответов" (accuracy) :

In [ ]:
sum(y_pred == y_test)

In [ ]:
sum(y_pred == y_test) / len(y_test)

In [ ]:
def accuracy_score(y_true, y_predict):
    '''input: y_true, y_predict
       returns accuracy'''
    assert y_true.shape[0] == y_predict.shape[0], \
        "the size of y_true must be equal to the size of y_predict"

    return sum(y_true == y_predict) / len(y_true)

In [ ]:
accuracy_score(y_test, y_pred)

Также качество классификации можно оценить при помощи матрицы ошибок (по строкам истинные классы, по столбцам предсказанные классы), отчета о классификации, показателя доли ошибок:

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

conf_mat=confusion_matrix(y_test,y_pred)
print(conf_mat)

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
print("Доля ошибок неправильной классификации:",round(np.mean(y_pred!=y_test),3))

## Выбор оптимальных параметров классификатора

При использовании классификаторов с параметрами возникает вопрос выбора оптимальных параметров классификатора.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

best_score = 0.0
best_k = -1
for k in range(1, 11):
    knn_clf = KNeighborsClassifier(n_neighbors=k)
    knn_clf.fit(X_train, y_train)
    score = knn_clf.score(X_test, y_test)
    if score > best_score:
        best_k = k
        best_score = score
        
print("Лучшее k =", best_k)
print("Лучшая оценка =", best_score)

Классификатор `KNeighborsClassifier` имеет параметр `weights`, который может принимать значения:

* `uniform`: единый вес, когда все точки в каждой окрестности имеют одинаковый вес
* `distance`: точки взвешиваются, обратно пропорционально расстоянию до них, в этом случае более близкие соседи точки будут иметь большее влияние, чем соседи, находящиеся дальше

Выберем лучший классификатор в зависимости от числа соседей `k` и параметра `weights`:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

best_score = 0.0
best_k = -1
best_method = ""
for method in ["uniform", "distance"]:
    for k in range(1, 11):
        knn_clf = KNeighborsClassifier(n_neighbors=k, weights=method)
        knn_clf.fit(X_train, y_train)
        score = knn_clf.score(X_test, y_test)
        if score > best_score:
            best_k = k
            best_score = score
            best_method = method
        
print("Лучший метод =", best_method)
print("Лучшее k =", best_k)
print("Лучшая оценка =", best_score)

Также у классификатора `KNeighborsClassifier` есть параметр `p`, который представляет собой степень в метрике Минковского:

$\rho\left(\mathbf{x},\mathbf{y}\right) = \left(\sum_{i}\left|x_{i}-y_{i}\right|^{p}\right)^{\frac{1}{p}}$

Можно выбрать лучший классификатор в зависимости от числа соседей `k` и параметра `p`:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

best_score = 0.0
best_k = -1
best_p = -1

for k in range(1, 11):
    for p in range(1, 6):
        knn_clf = KNeighborsClassifier(n_neighbors=k, weights="distance", p=p)
        knn_clf.fit(X_train, y_train)
        score = knn_clf.score(X_test, y_test)
        if score > best_score:
            best_k = k
            best_p = p
            best_score = score
        
print("Лучшее k =", best_k)
print("Лучшее p =", best_p)
print("Лучшая оценка =", best_score)

В scikit-learn имеется мощный инструмент для автоматического подбора параметров для моделей машинного обучения `GridSearchCV`. `GridSearchCV` находит наилучшие параметры путем обычного перебора: он создает модель для каждой возможной комбинации параметров. Важно отметить, что такой подход может быть весьма затратным по ресурсам.

In [ ]:
param_grid = [
    {
        'weights': ['uniform'], 
        'n_neighbors': [i for i in range(1, 11)]
    },
    {
        'weights': ['distance'],
        'n_neighbors': [i for i in range(1, 11)], 
        'p': [i for i in range(1, 6)]
    }
]

In [ ]:
knn_clf = KNeighborsClassifier()

In [ ]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(knn_clf, param_grid)

In [ ]:
%%time
grid_search.fit(X_train, y_train)

Параметры лучшего классификатора и значение оценки классификатора находится в следующих свойствах:

In [ ]:
grid_search.best_estimator_

In [ ]:
grid_search.best_score_

Для ускорения нахождения параметров лучшего классификатора можно использовать параллельные вычисления (`n_jobs=-1`):

In [ ]:
%%time
grid_search = GridSearchCV(knn_clf, param_grid, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

Важнейший параметр классификатора – количество соседей – можно оценить также методом "локтя":

In [ ]:
error_rate = []

for i in range(1,60):
    
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train,y_train)
    pred_i = knn.predict(X_test)
    error_rate.append(np.mean(pred_i != y_test))

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(range(1,60),error_rate,color='blue', linestyle='dashed', marker='o',
         markerfacecolor='red', markersize=8)
plt.title('Доля ошибок классификации для параметра K', fontsize=20)
plt.xlabel('параметр K',fontsize=15)
plt.ylabel('Доля ошибок',fontsize=15);

## Наивная байесовская классификация

Техника наивной байесовской классификации основывается на так называемой теореме Байеса и подходит, в частности, когда размерность входных данных высока. Несмотря на свою простоту наивная байесовская классификация часто превосходит по эффективности более сложные методы классификации. 

### Теорема Байеса

Алгоритм базируется на знаменитой теореме Байеса, в которой фигурируют условные вероятности. 

Условная вероятность - это вероятность наступления некоторого события при условии, что некоторое другое событие уже произошло. Зная условные вероятности, можно посчитать вероятность события, используя априорную информацию.  

Математически теорема Байеса состоит в следующем:

$${\displaystyle P(A\mid B)={\frac {P(B\mid A)\,P(A)}{P(B)}},}$$

где $A$ и $B$ – события и $P(B)\neq{0}$.

$P(A\mid B)$ – это условная вероятность: правдоподобие наступления события $A$ при условии, что произошло событие $B$.

$P(B\mid A)$ – это также условная вероятность: правдоподобие наступления события $B$ при условии, что произошло событие $A$.

$P(A)$ и $P(B)$ – это вероятности наступления $A$ и $B$ независимо друг от друга.

### Почему наивный байесовский и почему алгоритм быстр

Метод называют наивным байесовским, потому что расчет вероятностей для каждого события упрощен, чтобы сделать расчет легко выполнимым. Вместо того, чтобы пытаться вычислить значения каждого значения атрибута, они считаются условно независимыми при заданном целевом значении.

Это очень сильное предположение, которое маловероятно в реальных условиях (что признаки не связаны). Тем не менее, этот подход на удивление хорошо работает на различных наборах данных, в т.ч. в тех, в которых это предположение не выполняется.

Обучение происходит быстро, потому что необходимо рассчитать только вероятность каждого класса и вероятность каждого класса при различных входных значениях.

Вероятности классов - это просто количество записей каждого класса, деленное на общее количество записей. Условные вероятности - это количество появлений каждого значения признака для данного значения класса, деленное на количество записей с этим значением класса.

В качестве примера будем рассматривать набор данных о качестве вина 
(http://archive.ics.uci.edu/ml/datasets/Wine).

Считаем данные из набора и проведем их анализ:

In [ ]:
df = datasets.load_wine(as_frame=True).frame
df.head(10)

In [ ]:
df.info()

In [ ]:
df.describe()

Визуализируем диаграммы размаха в разбивке по классам:

In [ ]:
for c in df.columns[:-1]:
    df.boxplot(c,by='target',figsize=(7,4),fontsize=14)
    plt.title("{}\n".format(c),fontsize=16)
    plt.xlabel("Класс вина", fontsize=16)

Из диаграмм размаха следует, что некоторые признаки классифицируют классы вин достаточно хорошо. 
Например, показателям Alcalinity, Total Phenols или Flavonoids соответствуют диаграммы с хорошо отделимыми медианами, характеризующими классы вин.

Ниже визуализация разделения на классы по двум переменным:

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(df['proline'],df['flavanoids'],c=df['target'],edgecolors='k',alpha=0.8,s=100)
plt.grid(True)
plt.title("Диаграмма рассеяния двух признаков,\nдемонстрирующая корреляцию и разделение классов",fontsize=15)
plt.xlabel("proline",fontsize=15)
plt.ylabel("Flavanoids",fontsize=15);

Для того, чтобы проверить уровень корреляции между признаками, визуализируем матрицу корреляций:

In [ ]:
import warnings
warnings.filterwarnings("ignore")

def correlation_matrix(df):
    from matplotlib import pyplot as plt
    from matplotlib import cm as cm

    fig = plt.figure(figsize=(16,12))
    ax1 = fig.add_subplot(111)
    cmap = cm.get_cmap('jet', 30)
    cax = ax1.imshow(df.corr(), interpolation="nearest", cmap=cmap)
    ax1.grid(True)
    plt.title('Корреляции признаков в наборе данных качества вин\n',fontsize=15)
    labels=df.columns
    ax1.set_xticklabels(labels,fontsize=9)
    ax1.set_yticklabels(labels,fontsize=9)
    fig.colorbar(cax, ticks=[0.1*i for i in range(-11,11)])
    plt.show()

correlation_matrix(df.drop('target',axis=1))

Разбиение на обучающую и тестовую выборки:

In [ ]:
from sklearn.model_selection import train_test_split

test_size=0.3 # тестовая выборка 30%

In [ ]:
X = df.drop('target',axis=1)
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)

In [ ]:
X_train.shape

In [ ]:
X_train.head()

### Классификация методом GaussianNB()

Пусть дана переменная класса $y$ и вектор признаков от $x_1$ до $x_n$, тогда теорема Байеса утверждает следующее:

$$P(y \mid x_1, \dots, x_n) = \frac{P(y) P(x_1, \dots x_n \mid y)} {P(x_1, \dots, x_n)}$$
Используя (наивное) допущение независимости, получаем
$$P(x_i | y, x_1, \dots, x_{i-1}, x_{i+1}, \dots, x_n) = P(x_i | y),$$
для всех $i$, поэтому 
$$P(y \mid x_1, \dots, x_n) = \frac{P(y) \prod_{i=1}^{n} P(x_i \mid y)} {P(x_1, \dots, x_n)}$$

Так как вероятность $P(x_1, \dots, x_n)$ не зависит от $y$, можно воспользоваться следующим правилом классификации:
$$P(y \mid x_1, \dots, x_n) \propto P(y) \prod_{i=1}^{n} P(x_i \mid y)$$
$$\Downarrow$$ 
$$\hat{y} = \arg\max_y P(y) \prod_{i=1}^{n} P(x_i \mid y)$$

Здесь $P(y)$ – это относительная частота класса $y$ в обучающей выборке.

Распределение признаков принимается нормальным:

$$ P(x_i \mid y) = \frac{1}{\sqrt{2\pi\sigma^2_y}} \exp(-\frac{(x_i - \mu_y)^2}{2\sigma^2_y}) $$

Параметры $\sigma_y$ и $\mu_y$ определяются непосредственным расчетом.

In [ ]:
from sklearn.naive_bayes import GaussianNB

nbc = GaussianNB()
nbc.fit(X_train,y_train);

Прогнозирование меток классов дает следующий результат (зависящий от разбиения на обучающее и тестовое множества):

In [ ]:
y_pred = nbc.predict(X_test)
mislabel = np.sum((y_test!=y_pred))
print("Количество неправильно классифицированных точек из {} точек тестового множества равно {}".format(len(y_test),mislabel))

Можно воспользоваться следующим отчетом о классификации из sklearn:

In [ ]:
# from sklearn.metrics import classification_report
print("Отчет о классификации:\n")
print(classification_report(y_test,y_pred))

При помощи sklearn можно посчитать матрицу ошибок классификации:

In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix
cm = (confusion_matrix(y_test,y_pred))
cmdf = pd.DataFrame(cm,index=['Класс 1','Класс 2',' Класс 3'], columns=['Класс 1','Класс 2',' Класс 3'])
print("Матрица ошибок:\n")
cmdf

### Валидация модели

При валидации модели используем для оценки качества модели функцию `cross_val_score`. 

In [ ]:
from sklearn.model_selection import cross_val_score

### Кросс-валидация по K блокам

При кросс-валидации по `k` блокам данные делятся на `k` частей. Модель обучается на `k-1` блоках, при этом один блок откладывается для тестирования. Этот процесс повторяется, чтобы каждый из блоков был использован как тестовый. После завершения процесса получаем  оценку, включающую среднее значение и/или стандартное отклонение.

In [ ]:
# K-fold Cross-Validation
from sklearn.model_selection import KFold

kfold = KFold(n_splits=10)

results_kfold = cross_val_score(nbc, X, y, cv=kfold)
print("Доля верных ответов (accuracy): %.2f%%" % (results_kfold.mean()*100.0)) 

### Стратифицированная кросс-валидация по K блокам

Стратифицированный подход представляет собой разновидность кросс-валидации (перекрестной проверки), которая возвращает стратифицированные блоки, т. е. каждый блок содержит примерно такое же соотношение целевых меток, как и полные данные.

In [ ]:
# Stratified K-fold Cross-Validation
from sklearn.model_selection import StratifiedKFold

skfold = StratifiedKFold(n_splits=10)

results_skfold = cross_val_score(nbc, X, y, cv=skfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results_skfold.mean()*100.0, 
                                     results_skfold.std()*100.0))

### Кросс-валидация по отдельным объектам (Leave-One-Out)

LOOCV — это метод перекрестной валидации, в котором размер блока равен 1, а параметр `k` задается количеством записей в данных. Этот вариант полезен, когда обучающие данные имеют ограниченный размер.

In [ ]:
# Leave One Out Cross-Validation 
from sklearn.model_selection import LeaveOneOut

loocv = LeaveOneOut()

results_loocv = cross_val_score(nbc, X, y, scoring='recall', cv=loocv)
print("Полнота: %.2f%%" % (results_loocv.mean()*100.0))

### Повторяющиеся случайные разбиения на обучающую и тестовую выборки

Метод представляет собой гибрид традиционного разделения на обучающую и тестовую выборки и перекрестной проверки k-fold. 

In [ ]:
# Repeated Random Test-Train Splits
from sklearn.model_selection import ShuffleSplit

kfold_split = ShuffleSplit(n_splits=10, test_size=0.30)

results_split = cross_val_score(nbc, X, y, scoring='jaccard_macro', cv=kfold_split)
print("Коэффициент Жаккара: %.2f%% (%.2f%%)" % (results_split.mean()*100.0, 
                                                results_split.std()*100.0))

## Визуализация трехмерных данных

In [ ]:
from mpl_toolkits import mplot3d
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')

In [ ]:
fig = plt.figure(figsize=(12,10))
ax = plt.axes(projection='3d')
iris = datasets.load_iris()

xs = iris.data[:,0]
ys = iris.data[:,1]
zs = iris.data[:,2]
ax.scatter( xs, ys, zs, c=iris.target,s=100 )
ax.set_xlabel(iris.feature_names[0])
ax.set_ylabel(iris.feature_names[1])
ax.set_zlabel(iris.feature_names[2])
ax.view_init( azim=-120, elev=25 );

### Задание на лабораторную работу №4

#### Задание (10 баллов)

Для закрепленного за Вами варианта лабораторной работы:

1.	Считайте из заданного набора данных репозитария UCI значения трех признаков и метки класса. 

2.	Если среди меток класса имеются пропущенные значения, то удалите записи с пропущенными метками класса. Если в признаках имеются пропущенные значения, то замените пропущенные значения, используя метод, указанный в индивидуальном задании. Если количество различных меток классов превышает 4, то уменьшите количество классов.

3. Нормализуйте признаки набора данных методом, указанным в индивидуальном задании.

4.	Визуализируйте набор данных в виде точек трехмерного пространства с координатами, соответствующими трем признакам, отображая точки различных классов разными цветами. Подпишите оси и рисунок, создайте легенду набора данных. 

5.  Используя алгоритм снижения размерности данных, указанный в индивидуальном задании, уменьшите размерность признакового пространства до двух и визуализируйте набор данных в виде точек на плоскости, отображая точки различных классов разными цветами. Подпишите оси и рисунок, создайте легенду набора данных. 

6.  Используя разделение набора данных из двух признаков на обучающую и тестовую выборки в соотношении 75% на 25%, проведите классификацию тестовой выборки с помощью метода К ближайших соседей для различных значений К и определите оптимальное значение параметра К с минимальной долей ошибок. 

7.  Для найденного значения K постройте и выведите на экран отчет о классификации и матрицу ошибок.

8. Создайте модели классификации точек набора данных из трех признаков на базе следующих классификаторов:
* наивного байесовского классификатора  
* классификатора метода К ближайших соседей для значения К, определенного в п. 6.

9.  Используя указанный в индивидуальном задании метод валидации модели, проведите для набора данных из трех признаков оценку качества классификаторов из п. 8 относительно показателя, указанного в индивидуальном задании, и выведите на экран среднее значение и дисперсию этого показателя.

10. Определите, какой из классификаторов позволяет получить более высокое среднее значение показателя классификации, проведите классификацию точек набора данных этим классификатором и визуализируйте набор данных в виде точек трехмерного пространства с координатами, соответствующими трем признакам, отображая точки различных прогнозируемых классов разными цветами. Подпишите оси и рисунок, создайте легенду набора данных. 
